In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [2]:
spark = SparkSession.builder \
    .master("local[*]")\
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "1g") \
    .getOrCreate()

In [4]:
from pipeline_oriented_analytics.dataframe import CsvDataFrame, ParquetDataFrame
ParquetDataFrame('../data/processed/distance_matrix', spark).show(4)

+--------------+---------------+--------+
|pickup_cell_14|dropoff_cell_14|distance|
+--------------+---------------+--------+
|      89c25983|       89c25e61|    8.44|
|      89c25859|       89c25857|    0.55|
|      89c25ff3|       89c258ff|   10.01|
|      89c26655|       89c2599d|   18.74|
+--------------+---------------+--------+
only showing top 4 rows



In [10]:
from pipeline_oriented_analytics.pipe import Pipe, IF
from pipeline_oriented_analytics.transformer import *
from typing import List, Dict
from pipeline_oriented_analytics.dataframe import CsvDataFrame, ParquetDataFrame
from pipeline_oriented_analytics import Phase

phase = Phase.train
#phase = Phase.predict

variables = ['id', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
lables = ['trip_duration']
column_names = {'pickup_longitude': 'pickup_lon', 'pickup_latitude': 'pickup_lat', 'dropoff_longitude': 'dropoff_lon', 'dropoff_latitude': 'dropoff_lat', 'trip_duration': 'duration_sec'}
variable_types = {'id': 'string', 'pickup_datetime': 'timestamp', 'pickup_lon': 'double', 'pickup_lat': 'double', 'dropoff_lon': 'double', 'dropoff_lat': 'double'}
label_types = {'duration': 'int'}

if phase.is_predict():
    columns = variables
    column_types = variable_types
    data_path = '../data/raw/test.csv'
else: 
    columns = variables + lables
    column_types = {**variable_types, **label_types}
    data_path = '../data/raw/train.csv'

df = Pipe([
    SelectColumns(columns),
    RenameColumns(column_names),
    NormalizeColumnTypes(column_types),
    CellToken(8, 'pickup_lat', 'pickup_lon', 'pickup_cell_8'),
    CellToken(8, 'dropoff_lat', 'dropoff_lon', 'dropoff_cell_8'),
    CellToken(14, 'pickup_lat', 'pickup_lon', 'pickup_cell_14'),
    CellToken(14, 'dropoff_lat', 'dropoff_lon', 'dropoff_cell_14'),
    Join(['pickup_cell_14', 'dropoff_cell_14'], Join.Method.left, ParquetDataFrame('../data/processed/distance_matrix', spark)),
    DropColumns(inputCols=['pickup_lat', 'pickup_lon', 'dropoff_lon', 'dropoff_lat', 'pickup_cell_14', 'dropoff_cell_14']),
    #SaveToParquet(f'../data/processed/{phase.name}/inputs'),
]).transform(CsvDataFrame(data_path, spark))

print(f'Saved {df.count()} rows of {phase.name} inputs')

Saved 1458644 rows of train inputs


In [15]:
df.select('dropoff_cell_8').groupby('dropoff_cell_8').count().sort(f.desc('count')).count()

54

In [16]:
ParquetDataFrame('../data/processed/distance_matrix', spark).show()

+--------------+---------------+--------+
|pickup_cell_14|dropoff_cell_14|distance|
+--------------+---------------+--------+
|      89c25983|       89c25e61|    8.44|
|      89c25859|       89c25857|    0.55|
|      89c25ff3|       89c258ff|   10.01|
|      89c26655|       89c2599d|   18.74|
|      89c2599b|       89c25901|    2.75|
|      89c25859|       89c259c7|    3.09|
|      89c259a7|       89c25859|    2.75|
|      89c259ad|       89c25885|    3.86|
|      89c25853|       89c25a1d|    4.99|
|      89c25857|       89c25ed9|    5.85|
|      89c2f671|       89c2589d|    3.53|
|      89c2590f|       89c25979|    2.16|
|      89c25851|       89c2f67d|    7.99|
|      89c2665f|       89c25a29|   19.96|
|      89c258ef|       89c2598d|     5.3|
|      89c259ab|       89c259b9|     1.7|
|      89c25a11|       89c259af|    4.99|
|      89c258ff|       89c2585b|    1.62|
|      89c25853|       89c2590b|    2.58|
|      89c2599f|       89c2584d|     3.1|
+--------------+---------------+--

In [17]:
df.select('dropoff_cell_8').groupby('dropoff_cell_8').count().count()

54

In [18]:
df.select('dropoff_cell_8').dropDuplicates().count()

54

In [14]:
df.select('dropoff_cell_8').groupby('dropoff_cell_8').count().sort(f.desc('count')).show()

+--------------+-------+
|dropoff_cell_8|  count|
+--------------+-------+
|         89c25|1341333|
|         89c2f|  95207|
|         89c27|  19989|
|         89c29|   1742|
|         89c3b|    111|
|         89e83|     73|
|         89c2b|     42|
|         89c3d|     23|
|         89c31|     17|
|         89e81|     14|
|         89c23|     14|
|         89c2d|      9|
|         89e9d|      7|
|         89c39|      6|
|         89e7d|      5|
|         89c3f|      5|
|         89b7b|      4|
|         89b7d|      2|
|         89dd3|      2|
|         89e85|      2|
+--------------+-------+
only showing top 20 rows

